# VacationPy

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
clean_city_data = pd.read_csv("../output_data/cities.csv")
clean_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Geraldton,-28.7667,114.6000,64.40,77,90,24.16,AU,1612574521
1,Rikitea,-23.1203,-134.9692,78.76,70,11,12.75,PF,1612574215
2,Vila Velha,-20.3297,-40.2925,82.40,74,75,10.36,BR,1612574546
3,Riviere-au-Renard,48.9957,-64.3971,32.00,93,90,4.61,CA,1612574614
4,Kapaa,22.0752,-159.3190,71.60,56,1,5.75,US,1612574369
...,...,...,...,...,...,...,...,...,...
555,Pandan,14.0453,124.1694,80.24,85,0,9.19,PH,1612574573
556,San-Pédro,4.7485,-6.6363,78.37,93,14,6.71,CI,1612574744
557,Karakol,42.4907,78.3936,21.52,70,47,3.51,KG,1612574744
558,Parfen'yevo,58.4840,43.4088,-0.74,92,100,10.18,RU,1612574744


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
# configure gmaps with API key
gmaps.configure(api_key=g_key)

# setting locations and weight
clean_city_data = clean_city_data.dropna()

locations = clean_city_data[["Lat", "Lng"]].astype(float)
humidity = clean_city_data["Humidity"].astype(float)

# Create a heatmap layer
fig = gmaps.figure(zoom_level=1.5, center=(20,20))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 max_intensity=max(clean_city_data["Humidity"]))
    
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [32]:
# create my ideal weather conditions: max temperature < 85 degrees but > 70,
# humidity between 30-50%, and low change of cloudiness.
ideal_weather_df = clean_city_data.loc[(clean_city_data["Max Temp"] >= 70) & 
                                        (clean_city_data["Max Temp"] <= 85) & 
                                        (clean_city_data["Humidity"] >= 30) &
                                        (clean_city_data["Humidity"] <= 50) & 
                                        (clean_city_data["Cloudiness"] <= 20)]

# drop any rows will null values
ideal_weather_df = ideal_weather_df.dropna()    

# display dataframe
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,New Norfolk,-42.7826,147.0587,77.00,36,0,7.00,AU,1612574448
160,Tres Arroyos,-38.3739,-60.2798,72.54,30,0,7.47,AR,1612574650
215,Nishihara,26.1842,127.7558,71.60,43,20,5.75,JP,1612574628
337,Lakes Entrance,-37.8811,147.9810,80.01,45,3,15.99,AU,1612574691
355,Broken Hill,-31.9500,141.4333,73.40,43,0,16.11,AU,1612574517
373,Nānākuli,21.3906,-158.1547,77.00,47,1,17.27,US,1612574699
458,Acajutla,13.5928,-89.8275,82.40,47,0,2.30,SV,1612574539
547,Cervantes,-39.0500,-67.3833,84.20,34,0,3.44,AR,1612574741


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels within 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
# create new hotel dataframe
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]].copy()

# add a new column for Hotel Name and set empty parameters to store data
hotel_df["Hotel Name"] = ""

# set parameters to search for hotels within 5000 meters
parameters = {
    "type": "lodging",
    "radius": 5000,
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    # set parameters to loop through each city's coordinates
    lat = row["Lat"]
    lng = row["Lng"]
    parameters["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make API request
    response = requests.get(base_url, params=parameters).json()
    
    # extract search results
    results = response["results"]
    
    # store search results to Hotel Name column
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except:
        print("no hotel found")
    
hotel_df

,City,Country,Lat,Lng,Hotel Name
12,New Norfolk,AU,-42.7826,147.0587,The Shingles Riverside Cottages
160,Tres Arroyos,AR,-38.3739,-60.2798,Habitacion Triple con baño y entrada privada
215,Nishihara,JP,26.1842,127.7558,Minshuku Agaihama
337,Lakes Entrance,AU,-37.8811,147.9810,The Esplanade Resort & Spa
355,Broken Hill,AU,-31.9500,141.4333,Royal Exchange Hotel
373,Nānākuli,US,21.3906,-158.1547,Camp Pālehua
458,Acajutla,SV,13.5928,-89.8275,asi es
547,Cervantes,AR,-39.0500,-67.3833,Iglesia Evangelica Pentecostal Ministerio Peniel


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name:</dt><dd>{Hotel Name}</dd>
<dt>City:</dt><dd>{City}</dd>
<dt>Country:</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
pin_locations = hotel_df[["Lat", "Lng"]]

In [43]:
# Plot hotels on top of the humidity heatmap with pins containing Hotel Name, City, and Country.

# Create a marker_layer using the hotel list to fill the info box
fig = gmaps.figure(zoom_level=1.5, center=(20,20))
markers = gmaps.marker_layer(pin_locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure with hotel names on top of heat map
fig

Figure(layout=FigureLayout(height='420px'))